In [1]:
!python -m PyQt5.uic.pyuic -x pyqt_gui/gui.xml -o pyqt_gui/text_code1.py


In [3]:
import cv2
import sys
import mediapipe as mp
from helper import mixamo_helper as mmh
from helper import mediapipe_helper as mph
from helper import pyglm_helper as glmh
import glm
import json
from IPython.display import clear_output
from helper import mediapipe_to_mixamo as mtm

In [3]:
def avg_vec3(v1, v2):
    v3 = glm.vec3((v1.x + v2.x)* 0.5, 
                 (v1.y + v2.y)* 0.5, 
                 (v1.z + v2.z)* 0.5)
    return v3

In [4]:
mp_name_idx_map =mph.get_name_idx_map()
mm_mp_map = mmh.get_mixamo_name_mediapipe_name_map()
mm_name_idx_map  =mmh.get_mixamo_name_idx_map()
mp_idx_mm_idx_map = dict()



for mm_name in mm_mp_map.keys():
    mp_name = mm_mp_map[mm_name]
    mp_idx = mp_name_idx_map[mp_name]
    mm_idx = mm_name_idx_map[mm_name]
    mp_idx_mm_idx_map[mp_idx] = mm_idx



In [5]:
def detectPose2(mp_pose, mp_drawing, image, pose):
    # Create a copy of the input image.
    output_image = image.copy()
    
    image.flags.writeable = False

    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    image.flags.writeable = True

    # Initialize a list to store the detected landmarks.
    glm_list = [None]*26
    visibility_list = [None]*26
    hip2d = glm.vec3(0.0, 0.0, 0.0)

    if results.pose_world_landmarks:
        landmark = results.pose_world_landmarks.landmark

        glm_list[mmh.Mixamo.Hips] = glm.vec3(0.0, 0.0, 0.0)
        glm_list[mmh.Mixamo.Neck]  = avg_vec3(landmark[mp_pose.PoseLandmark.LEFT_SHOULDER], landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER])
        glm_list[mmh.Mixamo.Spine1] = avg_vec3(glm_list[mmh.Mixamo.Hips], glm_list[mmh.Mixamo.Neck])
        glm_list[mmh.Mixamo.Spine] = avg_vec3(glm_list[mmh.Mixamo.Hips], glm_list[mmh.Mixamo.Spine1])
        glm_list[mmh.Mixamo.Spine2] = avg_vec3(glm_list[mmh.Mixamo.Spine1], glm_list[mmh.Mixamo.Neck])
        glm_list[mmh.Mixamo.Head]  = avg_vec3(landmark[mp_pose.PoseLandmark.LEFT_EAR], landmark[mp_pose.PoseLandmark.RIGHT_EAR])
        glm_list[mmh.Mixamo.Neck].y *= -1
        glm_list[mmh.Mixamo.Neck].z *= -1
        glm_list[mmh.Mixamo.Spine].y *= -1
        glm_list[mmh.Mixamo.Spine].z *= -1
        glm_list[mmh.Mixamo.Spine1].y *= -1
        glm_list[mmh.Mixamo.Spine1].z *= -1
        glm_list[mmh.Mixamo.Spine2].y *= -1
        glm_list[mmh.Mixamo.Spine2].z *= -1
        glm_list[mmh.Mixamo.Head].y *= -1
        glm_list[mmh.Mixamo.Head].z *= -1
        for mp_idx in mp_idx_mm_idx_map.keys():
            mm_idx = mp_idx_mm_idx_map[mp_idx]
            glm_list[mm_idx] = glm.vec3(landmark[mp_idx].x, -landmark[mp_idx].y, -landmark[mp_idx].z)
            visibility_list[mm_idx] = landmark[mp_idx].visibility

    image_height, image_width, _ = image.shape

    # calculate hips
    if results.pose_landmarks:
        # Draw Pose landmarks on the output image.
        landmark = results.pose_landmarks.landmark
        hip2d_left = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        hip2d_right = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
        hip2d = avg_vec3(hip2d_left, hip2d_right)
        hip2d.y *= image_height
        hip2d.x *= image_width


    mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                              connections=mp_pose.POSE_CONNECTIONS)

    return output_image, glm_list, visibility_list, hip2d

In [6]:
# cap = cv2.VideoCapture(0)


In [7]:
model_file_name = '../sample_input_data/zom_model.json'
with open(model_file_name) as f:
        pix3d_json_object = json.load(f)
is_find, hip_node = glmh.find_hips(pix3d_json_object["node"])

root_node = glmh.ModelNode()

root_node.set_mixamo(hip_node, mm_name_idx_map)
root_node.normalize_spine()


In [9]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
min_visibility = 0.5
model_complexity = 1
pose_video = mp_pose.Pose(static_image_mode=False,
                        min_detection_confidence=min_visibility, 
                        model_complexity=model_complexity)

import time

cap = cv2.VideoCapture('C:\\Users\\No\\Desktop\\2022-04-29-10-33-07.mp4')

while cap.isOpened():
        clear_output()
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            # continue
            break
        image, glm_list, visibility_list, hip2d = detectPose2(mp_pose, mp_drawing, image, pose_video)
        if glm_list[0] != None:
            rv = []
            rg = []
            root_node.normalize(glm_list, mm_name_idx_map)
            root_node.calc_animation(glm_list, mm_name_idx_map)
            # root_node.get_vec_and_group_list(rv, rg, is_apply_tmp_transform= True)
            # glmh.draw_list([v for v in rv ], rg, range= 1.0, azim= 0)
            # time.sleep(0.1)

        # print(glm_list)
        cv2.imshow('MediaPipe Hands', image)
        # else:
            # cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
            
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()

KeyboardInterrupt: 

In [3]:
from helper import mediapipe_to_mixamo as mtm
import json
mpmanager = mtm.MediapipeManager()
mpmanager.is_hips_move = True
model_file_name = '../sample_input_data/zom_model.json'
image_file_name = "../sample_input_data/figure.gif"
_, json_file = mtm.mediapipe_to_mixamo(mpmanager, model_file_name, image_file_name)
with open("../sample_input_data/result.json", 'w') as f:
        json.dump(json_file, f, indent=2)

{'x': -12.882951736450195, 'y': -6.336294651031494, 'z': -1.0927687883377075}
{'x': -12.882951736450195, 'y': -6.336294651031494, 'z': -1.0927687883377075}


NameError: name 'json' is not defined